In [1]:
import pandas as pd
import altair as alt
import datetime

## Data preparation

In [2]:
df = pd.read_excel("2014-2020.xlsx", engine='openpyxl')

In [3]:
df = df.drop(['Unnamed: 12', 'Unnamed: 13'], axis=1)

In [4]:
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0


In [5]:
splited_hour_date = df['Час/Дата'].str.split('-', expand=True)
splited_date = splited_hour_date[1].str.split('.', expand=True)

In [6]:
df['Hour'] = splited_hour_date[0]
df['Day'] = splited_date[0]
df['Month'] = splited_date[1]
df['Year'] = splited_date[2]
df = df.drop('Час/Дата', axis=1)

In [7]:
df.head()

,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Hour,Day,Month,Year
0,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,24,31,12,2020
1,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,23,31,12,2020
2,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,22,31,12,2020
3,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,21,31,12,2020
4,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,20,31,12,2020


## 1. Як змінювалась структура генерації електроенергії за роками?


In [46]:
current_df = df.copy()

In [47]:
grouped_by_year = current_df.groupby("Year").sum()

In [48]:
grouped_by_year.reset_index(inplace=True)
grouped_by_year = grouped_by_year.drop(["CONSUMPTION", "GAES_PUMP", "UK_BLR_RUS", "UK_EURO", "UK_MLD"], axis=1)
grouped_by_year

,Year,AES,TEC,VDE,TES,GES,GAES_GEN
0,2014,88204418,14684731,1606386,68605877,7991074.0,839842
1,2015,87413763,12041074,1234936,50259819,4964478.0,1554857
2,2016,80762094,12841033,1093190,49879543,7113989.0,1621377
3,2017,85314258,12208062,1177032,44945303,8531952.0,1575099
4,2018,84351328,12261018,1863726,47747092,10072767.0,1564867
5,2019,83098265,12251850,4441801,44877587,6422154.0,1323192
6,2020,76210883,14536545,8124734,39504024,5998680.0,1561564


In [49]:
grouped_by_year = grouped_by_year.melt("Year")

In [50]:
alt.Chart(grouped_by_year).mark_line(
    point=alt.OverlayMarkDef()
).encode(
    x="Year",
    y=alt.Y("value", title="Electricity", axis=alt.Axis(labelExpr=f"datum.value + ' MW'")),
    color=alt.Color("variable", legend=alt.Legend(title="Source")),
    tooltip=["Year", "value"]
).properties(
   height=500, width=700
)

alt.Chart(...)

## 2. Як залежить споживання електроенергії від дня року та години доби?

In [35]:
current_df = df.copy()
current_df['Hour'] = current_df['Hour'].astype('int')
current_df = current_df.groupby(["Hour", "Month", "Day"]).mean().reset_index()

In [36]:
current_df['day_of_the_year'] = current_df['Month'] + '/' + current_df['Day']

In [37]:
current_df

,Hour,Month,Day,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,day_of_the_year
0,1,01,01,10574.857143,2338.142857,142.000000,4937.428571,822.428571,0.0,18202.428571,-116.285714,31.142857,-406.285714,-56.714286,01/01
1,1,01,02,10433.142857,2297.714286,143.571429,4564.142857,292.857143,0.0,16632.000000,-575.571429,-40.714286,-418.428571,-7.285714,01/02
2,1,01,03,10548.142857,2271.571429,152.142857,5242.714286,296.857143,0.0,17207.142857,-626.571429,-47.428571,-543.285714,-24.857143,01/03
3,1,01,04,10563.428571,2265.285714,148.857143,5172.428571,397.285714,0.0,17479.428571,-464.428571,20.000000,-535.000000,-23.285714,01/04
4,1,01,05,10514.142857,2295.000000,123.142857,5399.714286,397.571429,0.0,17663.571429,-504.142857,20.285714,-501.714286,-11.142857,01/05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,24,12,27,10640.857143,2144.714286,207.000000,5609.857143,202.285714,0.0,17845.428571,-246.285714,-123.428571,-472.714286,-34.142857,12/27
8780,24,12,28,10569.285714,2179.428571,194.285714,5810.428571,330.571429,0.0,17982.571429,-510.571429,28.000000,-492.714286,-40.142857,12/28
8781,24,12,29,10506.000000,2187.428571,302.714286,5445.142857,308.000000,0.0,17862.000000,-334.714286,18.142857,-439.571429,-49.571429,12/29
8782,24,12,30,10491.285714,2252.142857,197.142857,5822.857143,409.571429,0.0,18241.857143,-272.714286,-37.428571,-507.571429,-46.714286,12/30


In [38]:
chart_2 = alt.data_transformers.disable_max_rows()

alt.Chart(current_df).transform_filter(alt.FieldRangePredicate(field='Hour', range=[1, 24])).mark_rect().encode(
    x = alt.X('Hour:O', axis=alt.Axis(labelAngle=0)),
    y = alt.Y('day_of_the_year:O', axis=alt.Axis(title="Day of the year(Month/Day)")),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'plasma')),
    tooltip = ['day_of_the_year', 'Hour','CONSUMPTION']
).configure_legend(gradientLength=500).properties(width = 700, height = 1000)

alt.Chart(...)

## 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

In [41]:
current_df = df.copy()
current_df['Hour'] = current_df['Hour'].astype('int')

In [42]:
grouped_by_hour = current_df.groupby("Hour").mean()

In [43]:
grouped_by_hour.reset_index(inplace=True)
grouped_by_hour = grouped_by_hour.drop(["CONSUMPTION", "GAES_PUMP", "UK_BLR_RUS", "UK_EURO", "UK_MLD"], axis=1)
grouped_by_hour

,Hour,AES,TEC,VDE,TES,GES,GAES_GEN
0,1,9533.777865,1428.560813,164.178334,4883.335549,413.657020,0.267110
1,2,9530.698084,1424.868987,164.754009,4772.738365,325.804849,0.000000
2,3,9530.246774,1425.223309,164.704341,4722.442315,314.622214,0.000000
3,4,9531.818537,1423.511928,164.179898,4684.545561,289.973797,0.000000
4,5,9535.267110,1425.863512,165.059836,4697.965976,323.253813,0.000000
5,6,9537.201017,1426.097380,173.395385,4752.780211,307.224873,0.000000
6,7,9543.824795,1449.109894,194.382088,5138.824013,587.585061,4.705905
7,8,9549.525225,1489.098944,266.048885,5476.794290,720.086820,30.294486
8,9,9549.798983,1498.626124,389.411029,5740.241689,1047.793117,112.888932
9,10,9548.000782,1500.823230,529.519750,5897.132968,1088.394212,183.693000


In [44]:
grouped_by_hour = grouped_by_hour.melt("Hour")

In [45]:
alt.Chart(grouped_by_hour).mark_line(
    point=alt.OverlayMarkDef()
).encode(
    x=alt.X("Hour", title="Hour"),
    y=alt.Y("value", title="Electricity", axis=alt.Axis(labelExpr=f"datum.value + ' MW'")),
    color=alt.Color("variable", legend=alt.Legend(title="Source")),
    tooltip=["Hour", alt.Tooltip("value", title="")]
).properties(
   height=500, width=700
)

alt.Chart(...)

## 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?
 

In [22]:
current_df = df.copy()
current_df['Season'] = current_df['Month'].astype('int') %12 // 3 + 1
current_df['Season'] = current_df['Season'].replace({1: "Winter", 2: "Spring", 3: "Summer", 4: "Autumn"})
current_df['Month_order'] = current_df['Month'].astype('int') % 12
current_df['Month'] = current_df['Month'].astype('int').replace({1: "January", 2: "February", 3: "March", 4: "April", 
                                                                 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 
                                                                 10: "October", 11: "November", 12: "December"})
current_df['Hour'] = current_df['Hour'].astype('int')
current_df = current_df.groupby(["Hour", "Month", 'Season']).mean().reset_index()

In [23]:
current_df

,Hour,Month,Season,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Month_order
0,1,April,Spring,9221.100000,1265.738095,158.704762,4597.814286,706.333333,1.528571,14712.419048,-452.923810,-52.609524,-539.209524,-80.004762,4
1,1,August,Summer,8556.023041,779.400922,153.788018,4677.912442,320.216590,0.000000,13606.299539,-374.327189,11.032258,-357.783410,-61.354839,8
2,1,December,Winter,10712.294931,2154.133641,230.447005,5254.336406,284.069124,0.000000,17277.004608,-683.447005,-71.635945,-503.658986,-27.479263,0
3,1,February,Winter,11058.085859,2234.606061,178.090909,5010.601010,461.722222,0.000000,17505.383838,-648.005051,42.257576,-658.752525,-43.666667,2
4,1,January,Winter,10802.857143,2319.322581,165.161290,5420.267281,444.188940,0.000000,17855.262673,-587.216590,20.548387,-593.571429,-30.235023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,24,March,Spring,10203.202765,2030.364055,192.055300,5152.032258,625.161290,0.000000,17039.566820,-299.889401,-26.078341,-657.483871,-49.691244,3
284,24,May,Spring,8363.207373,913.396313,126.377880,5052.253456,783.889401,25.000000,14317.861751,-179.179724,-100.520737,-470.672811,-101.880184,5
285,24,November,Autumn,10373.238095,1868.647619,207.314286,5527.661905,254.757143,3.257143,17193.147619,-343.338095,-22.314286,-525.380952,-43.371429,11
286,24,October,Autumn,9623.894009,1234.829493,172.709677,5411.543779,196.198157,4.437788,15610.626728,-296.921659,-39.589862,-524.465438,-52.967742,10


In [24]:
winter = current_df[current_df['Season'] == 'Winter']
spring = current_df[current_df['Season'] == 'Spring']
summer = current_df[current_df['Season'] == 'Summer']
autumn = current_df[current_df['Season'] == 'Autumn']

In [25]:
winter

,Hour,Month,Season,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Month_order
2,1,December,Winter,10712.294931,2154.133641,230.447005,5254.336406,284.069124,0.000000,17277.004608,-683.447005,-71.635945,-503.658986,-27.479263,0
3,1,February,Winter,11058.085859,2234.606061,178.090909,5010.601010,461.722222,0.000000,17505.383838,-648.005051,42.257576,-658.752525,-43.666667,2
4,1,January,Winter,10802.857143,2319.322581,165.161290,5420.267281,444.188940,0.000000,17855.262673,-587.216590,20.548387,-593.571429,-30.235023,1
14,2,December,Winter,10707.036866,2147.603687,230.953917,5102.059908,218.336406,0.000000,16733.861751,-981.142857,-93.783410,-508.294931,-17.774194,0
15,2,February,Winter,11051.176768,2229.651515,174.757576,4923.575758,317.878788,0.000000,17005.257576,-931.868687,55.717172,-661.833333,-27.984848,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,23,February,Winter,11077.262626,2328.969697,182.181818,6000.626263,496.601010,7.904040,19289.994949,-8.994949,6.767677,-591.904040,-60.555556,2
268,23,January,Winter,10833.599078,2440.193548,170.525346,6423.718894,527.364055,13.165899,19647.133641,-9.202765,-19.640553,-549.580645,-61.788018,1
278,24,December,Winter,10707.304147,2181.391705,233.649770,5917.184332,282.643519,1.552995,18189.585253,-364.078341,-153.668203,-501.442396,-38.267281,0
279,24,February,Winter,11072.919192,2252.651515,181.176768,5651.207071,404.666667,1.616162,18368.343434,-349.843434,-7.505051,-645.373737,-57.343434,2


In [26]:
winter_chart = alt.Chart(winter).transform_filter(alt.FieldRangePredicate(field='Hour', range=[1, 24])).mark_rect().encode(
    x = 'Hour:O',
    y = alt.Y('Month:O', sort=alt.Sort(field='Month_order'), title="Winter"),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'plasma')),
    tooltip = ['Month', 'Hour','CONSUMPTION']
)

spring_chart = alt.Chart(spring).transform_filter(alt.FieldRangePredicate(field='Hour', range=[1, 24])).mark_rect().encode(
    x = 'Hour:O',
    y = alt.Y('Month:O', sort=alt.Sort(field='Month_order'), title="Spring"),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'plasma')),
    tooltip = ['Month', 'Hour','CONSUMPTION']
)

summer_chart = alt.Chart(summer).transform_filter(alt.FieldRangePredicate(field='Hour', range=[1, 24])).mark_rect().encode(
    x = 'Hour:O',
    y = alt.Y('Month:O', sort=alt.Sort(field='Month_order'), title="Summer"),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'plasma')),
    tooltip = ['Month', 'Hour','CONSUMPTION']
)

autumn_chart = alt.Chart(autumn).transform_filter(alt.FieldRangePredicate(field='Hour', range=[1, 24])).mark_rect().encode(
    x = 'Hour:O',
    y = alt.Y('Month:O', sort=alt.Sort(field='Month_order'), title="Autumn"),
    color = alt.Color('CONSUMPTION:Q', scale = alt.Scale(scheme = 'plasma')),
    tooltip = ['Month', 'Hour','CONSUMPTION']
)

alt.vconcat(
    winter_chart, spring_chart, summer_chart, autumn_chart
).configure_axisY(
    titleX=-60,
    labelPadding=0, 
    labelAlign='right'
).configure_legend(gradientLength=400) 


alt.VConcatChart(...)

## 5. Як змінюється споживання електроенергії впродовж тижня?

In [27]:
current_df = df.copy()

In [28]:
current_df['Day'] = current_df['Day'].astype('int')
current_df['Month'] = current_df['Month'].astype('int')
current_df['Year'] = current_df['Year'].astype('int')

In [29]:
weekdays = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}
def get_weekday(row):
    return weekdays[datetime.datetime(row['Year'], row['Month'], row['Day']).weekday()]

In [30]:
current_df['Weekday'] = current_df.apply(get_weekday, axis=1)

In [31]:
current_df = current_df.groupby('Weekday').mean().reset_index()

In [33]:
alt.Chart(current_df).mark_bar().encode(
    x=alt.X('Weekday:O', sort=list(weekdays.values()), axis=alt.Axis(labelAngle=0)),
    y=alt.Y("CONSUMPTION", title="Consumption", axis=alt.Axis(labelExpr=f"datum.value + ' MW'")),
    tooltip=["Weekday", "CONSUMPTION"]
).properties(width=700, height=400)

alt.Chart(...)

In [51]:
!jupyter nbconvert --to html HW2.ipynb  

[NbConvertApp] Converting notebook HW2.ipynb to html
[NbConvertApp] Writing 4021252 bytes to HW2.html
